# Jane Street Puzzle - March 2025 - Hall of Mirrors 3

## Problem Statement

<div class="center"><img src="images/mirrors_3.png" style="background-color: white;" class="dark_img" alt="", width="700" height="800"></div>

The perimeter of a 10-by-10 square field is surrounded by lasers pointing into the field. (Each laser begins half a unit from the edge of the field, as indicated by the •’s.)

Some of the lasers have numbers beside them. Place diagonal mirrors in some of the cells so that the product of the segment lengths of a laser’s path matches the clue numbers. (For instance, the segments for the “75” path in the example puzzle have lengths 5, 3, 5.) Mirrors may not be placed in orthogonally adjacent cells.

Once finished, determine the missing clue numbers for the perimeter, and calculate the sum of these clues for each side of the square. The answer to this puzzle is the product of these four sums.

## Solution

First, we generate all the valid mirror placements for each clue. In order to generate valid placements, we perform all the possible factor decompositions of the clues, excluding those where a factor is greater or equal to 10. Then, we generate all the permutations of those decompositions (all orderings of the factors). As the first and last factors can be 1, we also add all the permutations with starting with a one, ending with a one or both. In order to check if the factor ordering is valid, we try placing mirrors accordingly (all possibilities with the different mirror orientations) and check if the laser ends up on a missing clue. For each clue, we record all the valid mirror placements.

Once we have all the mirror placements for each clue, we use a backtracking algorithm to merge the grids. Once, we have the final grid, we simply connect the missing clues and compute their values. Finally, we sum the 4 perimeters and the answer is their product.

In [13]:
from itertools import permutations
import copy

def factor_combinations(num):
    def find_factors(n, start, path, result):
        """Recursive helper function to find factor combinations."""
        if n == 1 and len(path) > 1:  # Ensure at least two factors
            if all(f <= 10 for f in path):  # Filter combinations with factors > 10
                result.append(tuple(path[:]))  # Convert to tuple for immutability
            return
        for i in range(start, min(n + 1, 11)):  # Limit factor search to 10
            if n % i == 0:  # Check if 'i' is a factor
                path.append(i)
                find_factors(n // i, i, path, result)  # Continue factoring
                path.pop()  # Backtrack

    result = []
    find_factors(num, 2, [], result)

    # Generate permutations of each factorization and remove duplicates using a set
    unique_permutations = set()
    for factors in result:
        perms = set(permutations(factors))  # Get unique permutations
        unique_permutations.update(perms)  # Add to set

    unique_permutations = [list(perm) for perm in unique_permutations]

    for perm in unique_permutations.copy():
        unique_permutations.append([1] + perm)
        unique_permutations.append([1] + perm + [1])
        unique_permutations.append(perm + [1])

    if num <= 10:
        unique_permutations.append([1, num])
        if num != 1:
            unique_permutations.append([num, 1])
            unique_permutations.append([1, num, 1])

    return unique_permutations


In [14]:
res = []
grid = [[''] * 10 for _ in range(10)]
top = [0, 0, 112, 0, 48, 3087, 9, 0, 0, 1]
bottom = [2025, 0, 0, 12, 64, 5, 0, 405, 0, 0]
left = [0, 0, 0, 27, 0, 0, 0, 12, 225, 0]
right = [0, 4, 27, 0, 0, 0, 16, 0, 0, 0]


def place_mirrors(num, comb, pos, i, j, di, dj):

    if pos == len(comb):
        return

    new_i, new_j = i + comb[pos] * di, j + comb[pos] * dj

    if 0 <= new_i < 10 and 0 <= new_j < 10:
        prev_val = grid[new_i][new_j]
        # Laser from top to bottom
        if di == 1:
            # Skip if laser crosses mirror
            skip = False
            for k in range(i + 1, new_i):
                if grid[k][j] in ['/', '\\']:
                    skip = True
                    break
            # If laser does not cross a mirror
            if not skip:
                # Skip if / mirror cannot be placed
                if grid[new_i][new_j] not in ['0', '/']:
                    # Draw laser and mirror
                    for k in range(i + 1, new_i):
                        grid[k][j] = '0'
                    grid[new_i][new_j] = "\\"
                    # Go to next mirror placement
                    place_mirrors(num, comb, pos + 1, new_i, new_j, 0, 1)
                    # Remove mirror and laser
                    for k in range(i + 1, new_i):
                        grid[k][j] = ''
                    grid[new_i][new_j] = prev_val

                # Skip if \ mirror cannot be placed
                if grid[new_i][new_j] not in ['0', '\\']:
                    # Draw laser and mirror
                    for k in range(i + 1, new_i):
                        grid[k][j] = '0'
                    # Draw mirror
                    grid[new_i][new_j] = "/"
                    # Go to next mirror placement
                    place_mirrors(num, comb, pos + 1, new_i, new_j, 0, -1)
                    # Remove mirror and laser
                    for k in range(i + 1, new_i):
                        grid[k][j] = ''
                    grid[new_i][new_j] = prev_val
        # Laser from bottom to top
        elif di == -1:
            # Skip if laser crosses mirror
            skip = False
            for k in range(new_i + 1, i):
                if grid[k][j] in ['/', '\\']:
                    skip = True
                    break
            # If laser does not cross a mirror
            if not skip:
                # Skip if \ mirror cannot be placed
                if grid[new_i][new_j] not in ['0', '\\']:
                    # Draw laser and mirror
                    for k in range(new_i + 1, i):
                        grid[k][j] = '0'
                    grid[new_i][new_j] = "/"
                    # Go to next mirror placement
                    place_mirrors(num, comb, pos + 1, new_i, new_j, 0, 1)
                    # Remove mirror and laser
                    for k in range(new_i + 1, i):
                        grid[k][j] = ''
                    grid[new_i][new_j] = prev_val
                # Skip if / mirror cannot be placed
                if grid[new_i][new_j] not in ['0', '/']:
                    # Draw laser and mirror
                    for k in range(new_i + 1, i):
                        grid[k][j] = '0'
                    grid[new_i][new_j] = "\\"
                    # Go to next mirror placement
                    place_mirrors(num, comb, pos + 1, new_i, new_j, 0, -1)
                    # Remove mirror and laser
                    for k in range(new_i + 1, i):
                        grid[k][j] = ''
                    grid[new_i][new_j] = prev_val

        # Laser from left to right
        if dj == 1:
            # Skip if laser crosses mirror
            skip = False
            for k in range(j + 1, new_j):
                if grid[i][k] in ['/', '\\']:
                    skip = True
                    break
            # If laser does not cross a mirror
            if not skip:
                # Skip if \ mirror cannot be placed
                if grid[new_i][new_j] not in ['0', '/']:
                    # Draw laser and mirror
                    for k in range(j + 1, new_j):
                        grid[i][k] = '0'
                    grid[new_i][new_j] = "\\"
                    # Go to next mirror placement
                    place_mirrors(num, comb, pos + 1, new_i, new_j, 1, 0)
                    # Remove mirror and laser
                    for k in range(j + 1, new_j):
                        grid[i][k] = ''
                    grid[new_i][new_j] = prev_val
                # Skip if / mirror cannot be placed
                if grid[new_i][new_j] not in ['0', '\\']:
                    # Draw laser and mirror
                    for k in range(j + 1, new_j):
                        grid[i][k] = '0'
                    grid[new_i][new_j] = "/"
                    # Go to next mirror placement
                    place_mirrors(num, comb, pos + 1, new_i, new_j, -1, 0)
                    # Remove mirror and laser
                    for k in range(j + 1, new_j):
                        grid[i][k] = ''
                    grid[new_i][new_j] = prev_val
        # Laser from right to left
        elif dj == -1:
            # Skip if laser crosses mirror
            skip = False
            for k in range(new_j + 1, j):
                if grid[i][k] in ['/', '\\']:
                    skip = True
                    break
            # If laser does not cross a mirror
            if not skip:
                # Skip if / mirror cannot be placed
                if grid[new_i][new_j] not in ['0', '\\']:
                    # Draw laser and mirror
                    for k in range(new_j + 1, j):
                        grid[i][k] = '0'
                    grid[new_i][new_j] = "/"
                    # Go to next mirror placement
                    place_mirrors(num, comb, pos + 1, new_i, new_j, 1, 0)
                    # Remove mirror and laser
                    for k in range(new_j + 1, j):
                        grid[i][k] = ''
                    grid[new_i][new_j] = prev_val
                # Skip if \ mirror cannot be placed
                if grid[new_i][new_j] not in ['0', '/']:
                    # Draw laser and mirror
                    for k in range(new_j + 1, j):
                        grid[i][k] = '0'
                    grid[new_i][new_j] = "\\"
                    # Go to next mirror placement
                    place_mirrors(num, comb, pos + 1, new_i, new_j, -1, 0)
                    # Remove mirror and laser
                    for k in range(new_j + 1, j):
                        grid[i][k] = ''
                    grid[new_i][new_j] = prev_val

    # Handle case where laser need to reach final position
    elif pos == len(comb) - 1:
        # Generate copies of outside vectors for output
        new_top = top.copy()
        new_bottom = bottom.copy()
        new_left = left.copy()
        new_right = right.copy()
        if new_i == -1:
            if top[new_j] in [0, num]:
                # Skip if laser crosses mirror
                skip = False
                for k in range(new_i + 1, i):
                    if grid[k][j] in ['/', '\\']:
                        skip = True
                        break
                if not skip:
                    # Update outside vector
                    new_top[new_j] = num
                    # Draw laser
                    for k in range(new_i + 1, i):
                        grid[k][j] = '0'
                    res.append((comb[:], copy.deepcopy(grid), new_top, new_bottom, new_left, new_right))
                    # Remove laser
                    for k in range(new_i + 1, i):
                        grid[k][j] = ''
        elif new_i == 10:
            if bottom[new_j] in [0, num]:
                # Skip if laser crosses mirror
                skip = False
                for k in range(i + 1, new_i):
                    if grid[k][j] in ['/', '\\']:
                        skip = True
                        break
                if not skip:
                    # Update outside vector
                    new_bottom[new_j] = num
                    # Draw laser
                    for k in range(i + 1, new_i):
                        grid[k][j] = '0'
                    res.append((comb[:], copy.deepcopy(grid), new_top, new_bottom, new_left, new_right))
                    # Remove laser
                    for k in range(i + 1, new_i):
                        grid[k][j] = ''
        elif new_j == -1:
            if left[new_i] in [0, num]:
                # Skip if laser crosses mirror
                skip = False
                for k in range(new_j + 1, j):
                    if grid[i][k] in ['/', '\\']:
                        skip = True
                        break
                if not skip:
                    # Update outside vector
                    new_left[new_i] = num
                    # Draw laser
                    for k in range(new_j + 1, j):
                        grid[i][k] = '0'
                    res.append((comb[:], copy.deepcopy(grid), new_top, new_bottom, new_left, new_right))
                    # Remove laser
                    for k in range(new_j + 1, j):
                        grid[i][k] = ''
        elif new_j == 10:
            if right[new_i] in [0, num]:
                # Skip if laser crosses mirror
                skip = False
                for k in range(j + 1, new_j):
                    if grid[i][k] in ['/', '\\']:
                        skip = True
                        break
                if not skip:
                    # Update outside vector
                    new_right[new_i] = num
                    # Draw laser
                    for k in range(j + 1, new_j):
                        grid[i][k] = '0'
                    res.append((comb[:], copy.deepcopy(grid), new_top, new_bottom, new_left, new_right))
                    # Remove laser
                    for k in range(j + 1, new_j):
                        grid[i][k] = ''
        return

In [15]:
# Generate all possible grids for individual clues

all_grids = []

for i, num in enumerate(top):
    if num == 0:
        continue
    res = []
    for comb in factor_combinations(num):
        place_mirrors(num, comb, 0, -1, i, 1, 0)
    all_grids.append(copy.deepcopy(res))

for i, num in enumerate(bottom):
    if num == 0:
        continue
    res = []
    for comb in factor_combinations(num):
        place_mirrors(num, comb, 0, 10, i, -1, 0)
    all_grids.append(copy.deepcopy(res))

for i, num in enumerate(left):
    if num == 0:
        continue
    res = []
    for comb in factor_combinations(num):
        place_mirrors(num, comb, 0, i, -1, 0, 1)
    all_grids.append(copy.deepcopy(res))

for i, num in enumerate(right):
    if num == 0:
        continue
    res = []
    for comb in factor_combinations(num):
        place_mirrors(num, comb, 0, i, 10, 0, -1)
    all_grids.append(copy.deepcopy(res))

In [16]:
full_grids = []


def merge_grids(grid1, grid2, top1, top2, bottom1, bottom2, left1, left2, right1, right2):
    """ Merge two grids if they can be merged """
    new_grid = copy.deepcopy(grid1)
    new_top = copy.deepcopy(top1)
    new_bottom = copy.deepcopy(bottom1)
    new_left = copy.deepcopy(left1)
    new_right = copy.deepcopy(right1)

    # Check if laser and mirrors can be merged
    for i in range(len(grid1)):
        for j in range(len(grid2)):
            if grid1[i][j] != grid2[i][j]:
                if grid1[i][j] == '':
                    new_grid[i][j] = grid2[i][j]
                    if new_grid[i][j] in ['/', '\\']:
                        if (i > 0 and new_grid[i - 1][j] in ['/', '\\']) or (i < 9 and new_grid[i + 1][j] in ['/', '\\']) or (j > 0 and new_grid[i][j - 1] in ['/', '\\']) or (j < 9 and new_grid[i][j + 1] in ['/', '\\']):
                            return [], [], [], [], []
                elif grid2[i][j] != '':
                    return [], [], [], [], []


    # Check if outside clues can be merged
    for i in range(len(top1)):
        if top1[i] != top2[i]:
            if top1[i] == 0:
                new_top[i] = top2[i]
            elif top2[i] != 0:
                return [], [], [], [], []
    for i in range(len(bottom1)):
        if bottom1[i] != bottom2[i]:
            if bottom1[i] == 0:
                new_bottom[i] = bottom2[i]
            elif bottom2[i] != 0:
                return [], [], [], [], []
    for i in range(len(left1)):
        if left1[i] != left2[i]:
            if left1[i] == 0:
                new_left[i] = left2[i]
            elif left2[i] != 0:
                return [], [], [], [], []
    for i in range(len(right1)):
        if right1[i] != right2[i]:
            if right1[i] == 0:
                new_right[i] = right2[i]
            elif right2[i] != 0:
                return [], [], [], [], []

    return new_grid, new_top, new_bottom, new_left, new_right



def backtrack(i, curr_grid, curr_top, curr_bottom, curr_left, curr_right):
    """ Generate the final grid with backtracking. Each clue must contribute one grid. """

    if i == len(all_grids):
        full_grids.append((copy.deepcopy(curr_grid), curr_top.copy(), curr_bottom.copy(), curr_left.copy(), curr_right.copy()))
        return

    for _, grid, top, bottom, left, right in all_grids[i]:
        new_grid, new_top, new_bottom, new_left, new_right = merge_grids(curr_grid, grid, curr_top, top, curr_bottom, bottom, curr_left, left, curr_right, right)
        if not new_grid:
            continue
        backtrack(i + 1, new_grid.copy(), new_top.copy(), new_bottom.copy(), new_left.copy(), new_right.copy())


grid = [[''] * 10 for _ in range(10)]
top = [0, 0, 112, 0, 48, 3087, 9, 0, 0, 1]
bottom = [2025, 0, 0, 12, 64, 5, 0, 405, 0, 0]
left = [0, 0, 0, 27, 0, 0, 0, 12, 225, 0]
right = [0, 4, 27, 0, 0, 0, 16, 0, 0, 0]

backtrack(0, grid, top, bottom, left, right)

In [17]:
full_grids

[([['0', '0', '\\', '0', '0', '0', '\\', '0', '0', '\\'],
   ['0', '0', '0', '0', '0', '0', '0', '0', '\\', '0'],
   ['/', '0', '0', '0', '/', '0', '0', '/', '0', '0'],
   ['0', '0', '/', '0', '0', '0', '0', '0', '0', '\\'],
   ['0', '0', '0', '0', '0', '0', '/', '0', '0', '0'],
   ['0', '/', '0', '0', '\\', '0', '0', '\\', '0', '0'],
   ['/', '0', '\\', '0', '0', '/', '0', '0', '/', '0'],
   ['0', '0', '0', '\\', '0', '0', '0', '0', '0', '0'],
   ['0', '0', '0', '0', '/', '0', '/', '0', '/', '0'],
   ['/', '0', '0', '0', '0', '/', '0', '0', '0', '0']],
  [0, 0, 112, 0, 48, 3087, 9, 405, 4, 1],
  [2025, 225, 0, 12, 64, 5, 16, 405, 0, 3087],
  [27, 2025, 0, 27, 112, 0, 48, 12, 225, 0],
  [1, 4, 27, 9, 64, 27, 16, 0, 0, 5])]

In [18]:
## Fill the missing outside clues


grid, top, bottom, left, right = full_grids[0]


def fill_remaining(i, j, di, dj):
    prod = 1
    curr = 1
    while True:
        i += di
        j += dj
        if i in [-1, 10] or j in [-1, 10]:
            break
        if grid[i][j] == '0':
            curr += 1
        elif grid[i][j] == '/':
            prod *= curr
            curr = 1
            if di == 1:
                di, dj = 0, -1
            elif di == -1:
                di, dj = 0, 1
            elif dj == 1:
                di, dj = -1, 0
            elif dj == -1:
                di, dj = 1, 0
        elif grid[i][j] == '\\':
            prod *= curr
            curr = 1
            if di == 1:
                di, dj = 0, 1
            elif di == -1:
                di, dj = 0, -1
            elif dj == 1:
                di, dj = 1, 0
            elif dj == -1:
                di, dj = -1, 0

    prod *= curr
    if i == -1:
        top[j] = prod
    elif i == 10:
        bottom[j] = prod
    elif j == -1:
        left[i] = prod
    elif j == 10:
        right[i] = prod
    return prod



for k in range(len(top)):
    if top[k] == 0:
        top[k] = fill_remaining(-1, k, 1, 0)
    if bottom[k] == 0:
        bottom[k] = fill_remaining(10, k, -1, 0)
    if left[k] == 0:
        left[k] = fill_remaining(k, -1, 0, 1)
    if right[k] == 0:
        right[k] = fill_remaining(k, 10, 0, -1)

In [20]:
initial_sum_top = sum([0, 0, 112, 0, 48, 3087, 9, 0, 0, 1])
initial_sum_bottom = sum([2025, 0, 0, 12, 64, 5, 0, 405, 0, 0])
initial_sum_left = sum([0, 0, 0, 27, 0, 0, 0, 12, 225, 0])
initial_sum_right = sum([0, 4, 27, 0, 0, 0, 16, 0, 0, 0])

(sum(top) - initial_sum_top) * (sum(bottom) - initial_sum_bottom) * (sum(left) - initial_sum_left) * (sum(right) - initial_sum_right)

601931086080